In [1]:
import pandas as pd
import numpy as np
import seaborn as  sns

from sklearn.model_selection import train_test_split,GridSearchCV

from sklearn.pipeline import Pipeline
from sklearn.compose  import ColumnTransformer
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression

In [2]:
df = pd.read_csv('(Titanic)train.csv',usecols =['Sex','Age','Fare','Embarked','Parch','SibSp','Pclass','Survived'])
df.sample(6)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
203,0,3,male,45.5,0,0,7.2250,C
799,0,3,female,30.0,1,1,24.1500,S
572,1,1,male,36.0,0,0,26.3875,S
743,0,3,male,24.0,1,0,16.1000,S
798,0,3,male,30.0,0,0,7.2292,C
311,1,1,female,18.0,2,2,262.3750,C


In [3]:
X = df.drop(columns=['Survived'])
y =df['Survived']

In [4]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=45)

In [5]:
X_train.sample(6)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
889,1,male,26.0,0,0,30.0000,C
505,1,male,18.0,1,0,108.9000,C
550,1,male,17.0,0,2,110.8833,C
87,3,male,NaN,0,0,8.0500,S
775,3,male,18.0,0,0,7.7500,S
468,3,male,NaN,0,0,7.7250,Q


In [6]:
numerical_features = ['Age', 'Fare']
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_features = ['Embarked', 'Sex']
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('ohe',OneHotEncoder(handle_unknown='ignore'))
])

In [7]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

In [8]:
clf = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression())
])

In [9]:
from sklearn import set_config

set_config(display='diagram')
clf


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['Age', 'Fare']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('ohe',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Embarked', 'Sex'])])),
                ('classifier', LogisticRegression())])

In [10]:
param_grid = {
    'preprocessor__num__imputer__strategy': ['mean', 'median'],
    'preprocessor__cat__imputer__strategy': ['most_frequent', 'constant'],
    'classifier__C': [0.1, 1.0, 10, 100]
}

grid_search = GridSearchCV(clf, param_grid, cv=10)

In [11]:
grid_search.fit(X_train, y_train)

print(f"Best params:")
print(grid_search.best_params_)

Best params:
{'classifier__C': 1.0, 'preprocessor__cat__imputer__strategy': 'most_frequent', 'preprocessor__num__imputer__strategy': 'mean'}


In [12]:
print(f"Internal CV score: {grid_search.best_score_:.3f}")

Internal CV score: 0.771


In [13]:
import pandas as pd

cv_results = pd.DataFrame(grid_search.cv_results_)
cv_results = cv_results.sort_values("mean_test_score", ascending=False)
cv_results[['param_classifier__C','param_preprocessor__cat__imputer__strategy','param_preprocessor__num__imputer__strategy','mean_test_score']]


,param_classifier__C,param_preprocessor__cat__imputer__strategy,param_preprocessor__num__imputer__strategy,mean_test_score
4,1.0,most_frequent,mean,0.770931
5,1.0,most_frequent,median,0.770931
6,1.0,constant,mean,0.770931
7,1.0,constant,median,0.770931
8,10,most_frequent,mean,0.770931
9,10,most_frequent,median,0.770931
10,10,constant,mean,0.770931
11,10,constant,median,0.770931
12,100,most_frequent,mean,0.770931
13,100,most_frequent,median,0.770931
